# Explore Clean STU QQQ

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# takes about 1 minute to load
df_student = pd.read_csv('cleaned_data/clean_STU_QQQ.tsv', sep='\t', header=0, index_col=0, dtype={"LANGTEST_PAQ": "object"})
new_nan_values_count = df_student.isnull().sum().sum()
assert new_nan_values_count == 34114739, f"Failed check having {new_nan_values_count} NaNs"

In [3]:
# set index
df_student["CNTSTUID"] = df_student["CNTSTUID"].astype("string")
df_student = df_student.set_index(["CNTSTUID"])

# create datafrane with dropped columns
df = df_student.drop(columns=df_student.filter(regex='W_')).drop(columns=["SENWT", "VER_DAT"])
df = df.select_dtypes(exclude="object")
df = df[df.columns.drop(list(df.filter(regex='PV')))]

# add the PV1_ colums back
df_student = df.merge(
    df_student.filter(regex='^PV1[A-Z]+$'),
    how="inner",
    left_index=True,
    right_index=True,
)

df_student.shape

(613744, 1071)

In [4]:
df_student

,CNTSCHID,OECD,ADMINMODE,Option_CT,Option_FL,Option_ICTQ,Option_WBQ,Option_PQ,Option_TQ,Option_UH,...,PV1READ,PV1SCIE,PV1MCCR,PV1MCQN,PV1MCSS,PV1MCUD,PV1MPEM,PV1MPFS,PV1MPIN,PV1MPRE
CNTSTUID,,,,,,,,,,,,,,,,,,,,,
800001.0,800282.0,1,0,0,1,0,1,1,1,1,...,247.571,335.468,117.551,162.813,137.900,143.560,217.452,236.844,237.157,225.788
800002.0,800115.0,1,0,0,1,0,1,1,1,1,...,258.472,315.021,276.566,230.918,210.080,260.254,433.093,421.161,330.815,413.449
800003.0,800242.0,1,0,0,1,0,1,1,1,1,...,284.670,358.675,308.081,303.055,332.305,263.903,339.368,268.326,334.385,365.307
800005.0,800245.0,1,0,0,1,0,1,1,1,1,...,321.547,214.823,276.431,378.220,341.161,381.271,275.206,351.882,330.878,279.983
800006.0,800285.0,1,0,0,1,0,1,1,1,1,...,464.366,434.997,491.620,429.415,461.240,490.958,449.338,445.999,484.732,455.587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86007488.0,86000120.0,1,0,0,1,1,1,1,1,1,...,315.903,265.316,345.753,271.000,223.923,272.835,274.281,367.102,320.597,328.863
86007489.0,86000140.0,1,0,0,1,1,1,1,1,1,...,278.035,294.882,261.229,336.972,309.332,292.048,315.947,302.095,280.462,258.670
86007490.0,86000024.0,1,0,0,1,1,1,1,1,1,...,310.081,392.258,432.574,437.849,403.790,405.214,410.467,450.728,318.612,422.531


In [5]:
df_student.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 613744 entries, 800001.0 to 86007492.0
Data columns (total 1071 columns):
 #     Column       Non-Null Count   Dtype  
---    ------       --------------   -----  
 0     CNTSCHID     613744 non-null  float64
 1     OECD         613744 non-null  int64  
 2     ADMINMODE    613744 non-null  int64  
 3     Option_CT    613744 non-null  int64  
 4     Option_FL    613744 non-null  int64  
 5     Option_ICTQ  613744 non-null  int64  
 6     Option_WBQ   613744 non-null  int64  
 7     Option_PQ    613744 non-null  int64  
 8     Option_TQ    613744 non-null  int64  
 9     Option_UH    613744 non-null  int64  
 10    ST001D01T    613744 non-null  int64  
 11    ST003D02T    613744 non-null  int64  
 12    ST003D03T    601061 non-null  float64
 13    ST004D01T    613744 non-null  int64  
 14    ST250Q01JA   613744 non-null  int64  
 15    ST250Q02JA   613744 non-null  int64  
 16    ST250Q03JA   613744 non-null  int64  
 17    ST250Q04JA   613744

In [7]:
from sklearn.model_selection import train_test_split

# create train text split
X = df_student.drop(columns=["PV1MATH"])
y = df_student['PV1MATH'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print("Shape of Train Features: {}".format(X_train.shape))
print("Shape of Test Features: {}".format(X_test.shape))
print("Shape of Train Target: {}".format(y_train.shape))
print("Shape of Test Target: {}".format(y_test.shape))

Shape of Train Features: (368246, 1070)
Shape of Test Features: (245498, 1070)
Shape of Train Target: (368246,)
Shape of Test Target: (245498,)


## Data Processing

In [8]:
#  handle NaNs
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, add_indicator=True, strategy="constant", fill_value=-1)
imputer.fit(X_train)

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

## Train Model with LASSO Regression

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

### Use GridSearch CV to find best hyper parameter

In [ ]:
# parameters to be tested on GridSearchCV
params = {"alpha":np.arange(0.00001, 10, 500)}

# Number of Folds and adding the random state for replication
kf=KFold(n_splits=5,shuffle=True, random_state=42)

# Initializing the Model
lasso = Lasso()

# GridSearchCV with model, params and folds.
lasso_cv=GridSearchCV(lasso, param_grid=params, cv=kf)
lasso_cv.fit(X_train, y_train)
print("Best Params {}".format(lasso_cv.best_params_))

/Users/gatiaher/Code/pisa_oecd_analysis/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.390e+08, tolerance: 3.056e+05
  model = cd_fast.enet_coordinate_descent(
/Users/gatiaher/Code/pisa_oecd_analysis/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.388e+08, tolerance: 3.058e+05
  model = cd_fast.enet_coordinate_descent(
/Users/gatiaher/Code/pisa_oecd_analysis/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check t

### Use Lasso Regressor to rank predictive features

In [ ]:
names=X_train.columns
print("Column Names: {}".format(names.values))

# calling the model with the best parameter
lasso1 = Lasso(alpha=0.00001)
lasso1.fit(X_train, y_train)

# Using np.abs() to make coefficients positive.  
lasso1_coef = np.abs(lasso1.coef_)